### This script converts 3d gaussian ply file to have white foreground colour

In [1]:
import sys
sys.dont_write_bytecode = True

import numpy as np 
from plyfile import PlyData, PlyElement

### Constants

In [2]:
GAUSSIAN_FILE = './outputs/multiView_Text_random.ply' 
OUTPUT_FILE = '../gsplat_output/point_cloud/iteration_30000/point_cloud.ply' 

### Modify and save to new file

In [3]:
ply = PlyData.read(GAUSSIAN_FILE)

# Access the vertex data as a numpy structured array
vertices = np.array(ply['vertex'].data)

# Modify the color properties to white
# Assuming the values are normalized between 0 and 1
vertices['f_dc_0'] = np.ones(len(vertices), dtype=np.float32)
vertices['f_dc_1'] = np.ones(len(vertices), dtype=np.float32)
vertices['f_dc_2'] = np.ones(len(vertices), dtype=np.float32)

# set all other SH params to 0 to clear view-dependent colouring
for i in range(45):
    vertices[f'f_rest_{i}'] = np.zeros(len(vertices), dtype=np.float32)

# Create a new PlyElement for the modified vertices
updated_vertices = PlyElement.describe(vertices, 'vertex')

# Save the modified PLY data back to a new file
ply = PlyData([updated_vertices], text=ply.text)
ply.write(OUTPUT_FILE)



### Render Segmented Gaussians

In [4]:
GAUSSIAN_PATH = "D:/Projects/GaussianSplat/gaussian-splatting"
CURRENT_PATH = "D:/3D Sensing/GitHubRepo/sam_gsplat" 

In [5]:
%cd {GAUSSIAN_PATH} 

D:\Projects\GaussianSplat\gaussian-splatting


c:\ProgramData\anaconda3\envs\dust3r\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
!python render.py -m "D:\3D Sensing\GitHubRepo\gsplat_output" -s "D:\3D Sensing\GitHubRepo\gsplat_model" --data_device cuda

Looking for config file in D:\3D Sensing\GitHubRepo\gsplat_output\cfg_args
Config file found: D:\3D Sensing\GitHubRepo\gsplat_output\cfg_args
Rendering D:\3D Sensing\GitHubRepo\gsplat_output
Loading trained model at iteration 30000 [26/04 12:13:05]

Reading camera 1/251
Reading camera 2/251
Reading camera 3/251
Reading camera 4/251
Reading camera 5/251
Reading camera 6/251
Reading camera 7/251
Reading camera 8/251
Reading camera 9/251
Reading camera 10/251
Reading camera 11/251
Reading camera 12/251
Reading camera 13/251
Reading camera 14/251
Reading camera 15/251
Reading camera 16/251
Reading camera 17/251
Reading camera 18/251
Reading camera 19/251
Reading camera 20/251
Reading camera 21/251
Reading camera 22/251
Reading camera 23/251
Reading camera 24/251
Reading camera 25/251
Reading camera 26/251
Reading camera 27/251
Reading camera 28/251
Reading camera 29/251
Reading camera 30/251
Reading camera 31/251
Reading camera 32/251
Reading camera 33/251
Reading camera 34/251
Reading cam


Rendering progress: 100%|██████████| 219/219 [07:24<00:00,  2.03s/it]

Rendering progress: 100%|██████████| 32/32 [01:31<00:00,  2.87s/it]


In [7]:
%cd {CURRENT_PATH}

D:\3D Sensing\GitHubRepo\sam_gsplat


c:\ProgramData\anaconda3\envs\dust3r\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Create Folder with Ground Truths and Renders

The ground truth segmentation file names are all over the place, but they do have the file number that references the same image as the render. Thus, here we are just preparing the results in a common folder such that we can easily calculate metrics on them in the main pipeline files.

In [12]:
import os  
import shutil 
import cv2

TRUCK_SPINNERF_DATA = 'D:/3D Sensing/GitHubRepo/truck'  
DEST_DIR = '../renders'
RENDERED_PATH = 'D:/3D Sensing/GitHubRepo/gsplat_output/allRenders/ours_30000/renders'

# Create the destination directory if it doesn't exist
if not os.path.exists(DEST_DIR):
    os.makedirs(os.path.join(DEST_DIR, 'gt'))  
    os.makedirs(os.path.join(DEST_DIR, 'renders'))

for filename in os.listdir(TRUCK_SPINNERF_DATA):
    # Check if the file name contains 'pseudo'
    if 'pseudo' in filename:
        # Construct the full file path
        src_file = os.path.join(TRUCK_SPINNERF_DATA, filename)
        
        # Extract the 6 digit number after the first underscore in the file name
        new_filename = filename.split('_')[1][:6] + '.png' 

        # Construct the destination ground file path
        dst_file = os.path.join(DEST_DIR, 'gt' , new_filename) 

        # Construct the destination render file path 
        rend_file = os.path.join(RENDERED_PATH, new_filename) 

        # Create binary mask out of the gt and renders 
        gt_img = cv2.imread(src_file)
        rend_img = cv2.imread(rend_file)  

        gray_rend = cv2.cvtColor(rend_img, cv2.COLOR_BGR2GRAY)  
        _, binary_rend = cv2.threshold(gray_rend, int(0.6 * 255), 255, cv2.THRESH_BINARY) 

        gray_gt = cv2.cvtColor(gt_img, cv2.COLOR_BGR2GRAY)  
        _, binary_gt = cv2.threshold(gray_gt, int(0.6 * 255), 255, cv2.THRESH_BINARY)

        dst_file2 = os.path.join(DEST_DIR, 'renders' , new_filename)

        # Copy and rename the file
        cv2.imwrite(dst_file, binary_gt) 
        cv2.imwrite(dst_file2, binary_rend)    